#  Shex Creation Help

This tool will help to generate SheX for a given entity (Example E130.ipynb written by Tom Baker)

1. Obtaining suggestions for a given item

Given an item identifier (Q-number) on Wikidata,we would like to know the properties suggested by Wikidata. Wikidata has a suggestions operations for this purpose. 

2. Get properties of instances of an item

Given an item identifier (Q-number) on Wikidata, we would like to know the properties used by instances of this item

### Acknowledgements
Andra Waagmeester, Martin 

In [68]:
import requests

baseurl="https://www.wikidata.org/w/api.php?action=wbsgetsuggestions&format=json&limit=50&entity="

def get_property_suggestions(entity):
    """
    Returns string of property suggestion

    Parameters:
        entity (str): identifier of an identity

    Returns:
        (list): list of properties

    """
    
    propertylist = []
    url = baseurl + entity
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for prop in results["search"]:
            propertylist.append("wdt:" + prop["id"])
            
    return(propertylist)

In [84]:
l = get_property_suggestions("Q901")
print(l)

['wdt:P17']


In [70]:
from urllib.parse import quote
def get_currently_used_properties(entityid):
    propertylist = []
    query = """
       SELECT DISTINCT ?property
       {
          ?entity wdt:P31 wd:entityid;
                  ?prop [].
          ?property wikibase:directClaim ?prop.
       }"""
    baseurl = "https://query.wikidata.org/sparql?query="
    formatstr = "&format=json"

    query = query.replace("entityid", entityid)
    url = baseurl+quote(query)+formatstr
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for prop in results["results"]["bindings"]:
            propertylist.append(prop["property"]["value"].replace("http://www.wikidata.org/entity/","wdt:"))
    
    return(propertylist)

In [83]:
def get_properties(entity):
    propertylist = []
    propertylist.extend(get_property_suggestions(entity))
    propertylist.extend(get_currently_used_properties(entity))
                        
    return list(set(propertylist))

print(get_properties("Q901"))

['wdt:P279', 'wdt:P373', 'wdt:P69', 'wdt:P425', 'wdt:P1442', 'wdt:P31', 'wdt:P1559', 'wdt:P7148', 'wdt:P135', 'wdt:P4742', 'wdt:P802', 'wdt:P1053', 'wdt:P21', 'wdt:P19', 'wdt:P244', 'wdt:P1412', 'wdt:P734', 'wdt:P17', 'wdt:P2521', 'wdt:P119', 'wdt:P570', 'wdt:P569', 'wdt:P460', 'wdt:P512', 'wdt:P101', 'wdt:P1343', 'wdt:P691', 'wdt:P1153', 'wdt:P803', 'wdt:P185', 'wdt:P2163', 'wdt:P4252', 'wdt:P2650', 'wdt:P172', 'wdt:P496', 'wdt:P1066', 'wdt:P227', 'wdt:P18', 'wdt:P1960', 'wdt:P4613', 'wdt:P20', 'wdt:P40', 'wdt:P166', 'wdt:P22', 'wdt:P106', 'wdt:P463', 'wdt:P735', 'wdt:P108', 'wdt:P27', 'wdt:P213', 'wdt:P39', 'wdt:P1801', 'wdt:P214']


In [82]:
def get_property_datatype(entityid):
    csvresult =[]
    propertylist = get_properties(entityid)
    propertyliststr = ' '.join(propertylist) 
    query = """
    SELECT DISTINCT ("entityid" as ?entity) ?property ?datatype
    {
       VALUES ?prop {propertylist}.
       ?property wikibase:directClaim ?prop;
            wikibase:propertyType ?datatype.
    }"""
    
    baseurl = "https://query.wikidata.org/sparql?query="
    formatstr = "&format=json"

    query = query.replace("entityid", entityid)
    query = query.replace("propertylist", propertyliststr)
    url = baseurl+quote(query)+formatstr
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for result in results["results"]["bindings"]:
            csvresult.append(result["entity"]["value"]+ ", " +
                   result["property"]["value"].replace("http://www.wikidata.org/entity/", "wdt:")+ ", " +
                   result["datatype"]["value"])
    return(csvresult)
        
print('\n'.join(get_property_datatype("Q901")))

Q901, wdt:P279, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P425, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P460, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P463, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P512, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P735, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P734, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P803, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P802, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P1066, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P1343, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P1412, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P2650, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P17, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P19, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P21, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P20, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P22, http://wikiba.se/ontology#WikibaseItem
Q901, wdt:P